In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key='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'
)


response=llm.invoke("The first person to walk on the moon?")
print(response.content)

The first person to walk on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, during the Apollo 11 mission.


In [19]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-39013?from=job%20search%20funnel")
loader.session.verify = False  # Bypass SSL verification
page=loader.load().pop().page_content


C:\Users\saleh\anaconda3\envs\work\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobs.nike.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [20]:
print(page)

Apply for Senior Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySenior Software EngineerBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does

In [21]:
from langchain_core.prompts import PromptTemplate


prompt_extract=PromptTemplate.from_template(
    '''
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRCTION:
    The scrapped text is from the career's page of a website.
    Your job is to extract the job postings and return them in a json format containing the
    following keys: `title`, `minimum_qualifications`,`preffered_qualifications`,`about`,`location`,`company`,`skills`.
    Only include the first data
    Only return the valid JSON.
    ### VALID JSON ONLY (NO PREAMBLE):
    '''
    
)

chain_extract=prompt_extract|llm

res=chain_extract.invoke(input={'page_data':page})
print(res.content)

```
{
  "title": "Senior Software Engineer",
  "minimum_qualifications": [
    "Bachelor’s degree in Computer Science or IT related discipline or any acceptable combination of relevant education, training, and experience",
    "Strong abilities in Python (preferred) or similar object-oriented programming languages",
    "Intermediate skills in JavaScript front end-web frameworks (e.g. Angular, React)",
    "Knowledge of data structures and algorithms for efficiently and effectively managing large data sets",
    "Experience using serverless data warehouses (e.g. Athena) and NoSQL databases (e.g. DynamoDB)",
    "Tangible experience creating, delivering, and documenting GraphQL and RESTful APIs",
    "Cloud infrastructure development experience using public cloud providers, e.g., AWS (our preferred choice), Azure, GCP",
    "Practical understanding of infrastructure-as-code technologies (e.g., Terraform, CloudFormation, CircleCI), continuous integration & deployment (CI/CD), source cont

In [22]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'title': 'Senior Software Engineer',
 'minimum_qualifications': ['Bachelor’s degree in Computer Science or IT related discipline or any acceptable combination of relevant education, training, and experience',
  'Strong abilities in Python (preferred) or similar object-oriented programming languages',
  'Intermediate skills in JavaScript front end-web frameworks (e.g. Angular, React)',
  'Knowledge of data structures and algorithms for efficiently and effectively managing large data sets',
  'Experience using serverless data warehouses (e.g. Athena) and NoSQL databases (e.g. DynamoDB)',
  'Tangible experience creating, delivering, and documenting GraphQL and RESTful APIs',
  'Cloud infrastructure development experience using public cloud providers, e.g., AWS (our preferred choice), Azure, GCP',
  'Practical understanding of infrastructure-as-code technologies (e.g., Terraform, CloudFormation, CircleCI), continuous integration & deployment (CI/CD), source control (e.g. GitHub), and buil

In [23]:
import pandas as pd

df=pd.read_csv('my_portfolio.csv')

In [24]:
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [25]:
import uuid
import chromadb

client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas={'links':row['Links']},
                       ids=[str(uuid.uuid4())]
                      )



In [26]:
job=json_res
job['preferred_qualifications']

['Abilities to create elegant user experiences',
 'Practical experience in cloud cost optimization techniques and best practices',
 'Knowledge of cloud provider billing practices',
 'A basic understanding of FinOps practices',
 'Experience with big room planning processes such as Scaled Agile Framework (SAFe)']

In [27]:
links=collection.query(query_texts=job['skills'],n_results=1).get('metadatas',[])
links

[[{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}]]

In [28]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for NIKE, Inc.

Dear Hiring Manager,

I came across the Senior Software Engineer position at NIKE, Inc. and was impressed by the company's commitment to innovation and excellence. As a Business Development Executive at AtliQ, I'd like to introduce our company as a potential partner in fulfilling your software engineering needs.

At AtliQ, we specialize in providing AI and software consulting services that cater to the unique requirements of businesses like NIKE, Inc. Our team of experts has extensive experience in developing scalable, efficient, and cost-effective solutions that drive business growth.

Our capabilities align perfectly with the requirements of the Senior Software Engineer position. We have expertise in:

* Python and JavaScript development
* Front-end web frameworks like Angular and React
* Data structures and algorithms for managing large data sets
* Serverless data warehouses like Athena and NoSQL databases like DynamoDB
